<a href="https://colab.research.google.com/github/Arnes1997/product_category_classifier/blob/main/notebook/category_classifier_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛒 Product Category Classifier

Ovaj projekat razvijen je s ciljem **automatske klasifikacije proizvoda po kategorijama** na osnovu njihovih naslova i dodatnih atributa.  
Koristi se **machine learning model** obučen na stvarnom skupu podataka sa više od 30.000 proizvoda različitih vrsta (telefoni, kamere, mikrotalasne, frižideri i dr.).

---

## 📊 Opis skupa podataka

Ulazni skup podataka (`products.csv`) sadrži sledeće kolone:

| Kolona | Opis |
|--------|------|
| **Product ID** | Jedinstveni identifikator proizvoda |
| **Product Title** | Naziv proizvoda (npr. “Samsung Galaxy A52 128GB”) |
| **Merchant ID** | Identifikator prodavca |
| **Category Label** | Kategorija proizvoda (ciljna varijabla) |
| **Product Code** | Interni kod proizvoda |
| **Number of Views** | Broj pregleda proizvoda |
| **Merchant Rating** | Ocena prodavca |
| **Listing Date** | Datum postavljanja proizvoda |

1. **Učitavanje i istraživanje podataka (Exploratory Data Analysis)**  
   - Provjera jedinstvenih vrijednosti.  
   - Provera null vrednosti.  
   - Pregled osnovnih statistika i strukture podataka.  

In [11]:
# Ucitavanje potrebne biblioteke
import pandas as pd

# Ucitavanje products.csv fajla u dataframe
df = pd.read_csv("https://raw.githubusercontent.com/Arnes1997/product_category_classifier/refs/heads/main/data/products.csv")

# Pregled strukture podataka
print("*" * 100)
print("*** Kolone i tipovi podataka ***")
print(df.info())
print("*" * 100)
print("*** Prvih 5 redova ***")
print(df.head())
print("*" * 100)

# Pregled jedinstvenih vrijednosti
print("*** Jedinstvene vrijednosti u koloni Category Label ***")
print(df[' Category Label'].value_counts())
print("*" * 100)

# Provjera da li postoje null vrijednosti u df
print("*** Null vrijednosti ***")
print(df.isnull().sum())
print("*" * 100)



****************************************************************************************************
*** Kolone i tipovi podataka ***
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35311 entries, 0 to 35310
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   product ID       35311 non-null  int64  
 1   Product Title    35139 non-null  object 
 2   Merchant ID      35311 non-null  int64  
 3    Category Label  35267 non-null  object 
 4   _Product Code    35216 non-null  object 
 5   Number_of_Views  35297 non-null  float64
 6   Merchant Rating  35141 non-null  float64
 7    Listing Date    35252 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 2.2+ MB
None
****************************************************************************************************
*** Prvih 5 redova ***
   product ID                                      Product Title  Merchant ID  \
0           1             

2. **Čišćenje i priprema podataka (Data Cleaning)**  
   - Uklanjanje praznih redova.  
   - Uklanjanje nepotrbnih kolona.

In [22]:
# Zadrzava samo kolone product title i category label za treniranje modela
df = df[['Product Title', ' Category Label']]

# Ukloni redove koji imaju null u koloni 'Category Label' i 'Product Title'
df = df.dropna(subset=[' Category Label'])
df = df.dropna(subset=['Product Title'])

# Provjera da li je ociscena kolona Category Label
print("Null vrijednosti nakon ciscenja kolone Category Label: ", df[' Category Label'].isnull().sum())

# Provjera da li je ociscena kolona Product Title
print("Null vrijednosti nakon ciscenja kolone Product Title: ", df['Product Title'].isnull().sum())

Null vrijednosti nakon ciscenja kolone Category Label:  0
Null vrijednosti nakon ciscenja kolone Product Title:  0


3. **Inženjering karakteristika (Feature Engineering)**  
   Pored tekstualnih informacija iz naziva proizvoda, dodate su i numeričke osobine koje pomažu modelu da bolje razlikuje kategorije:  
   - `title_length` – dužina naslova (broj karaktera)  
   - `num_words` – broj riječi u naslovu  
   - `has_digits` – da li naslov sadrži brojeve  
   - `has_uppercase` – da li postoje riječi napisane velikim slovima  

In [23]:
# Ucitavanje potrebne biblioteke
import re

# Izračunava dužinu naslova proizvoda (broj karaktera)
df['title_length'] = df['Product Title'].apply(len)

# Izračunava broj riječi u naslovu proizvoda
df['num_words'] = df['Product Title'].apply(lambda x: len(x.split()))

# Provjerava da li naslov sadrži brojeve (0 ili 1)
df['has_digits'] = df['Product Title'].apply(lambda x: int(bool(re.search(r'\d', x))))

# Provjerava da li naslov sadrži barem jednu riječ napisanu velikim slovima
df['has_uppercase'] = df['Product Title'].apply(lambda x: int(any(w.isupper() for w in x.split())))

# Prikaz kolona
print("Kolone nakon (Feature Engineering):")
df.columns

Kolone nakon (Feature Engineering):


Index(['Product Title', ' Category Label', 'title_length', 'num_words',
       'has_digits', 'has_uppercase'],
      dtype='object')

4. **Vektorizacija teksta (Text Representation)**  
   - Tekstualni naslovi su konvertovani u numerički oblik korišćenjem **TF-IDF** (Term Frequency – Inverse Document Frequency).  
   - Korišćeni su **unigrami i bigrami** (1-2 reči po nizu) i maksimalno 5000 karakteristika.
   - Podjela podataka na trening i test skup (80% trening, 20% test)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Kreiramo TF-IDF vektorizator sa parametrima
text_transformer = TfidfVectorizer(max_features=5000, ngram_range=(1,2), stop_words='english')
num_transformer = StandardScaler()

# Kombinovanje TF-IDF i numeričkih značajki u ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('text', text_transformer, 'Product Title'),
        ('num', num_transformer, ['title_length', 'num_words', 'has_digits', 'has_uppercase'])
    ]
)

# Podjela podataka na trening i test
X = df.drop(' Category Label', axis=1)
y = df[' Category Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



5. **Treniranje i testiranje modela (Model Training & Testing)**  
   - Ispitano je više algoritama: *Logistic Regression, Random Forest, SVC*.  
   - Korišćene metrike:  
     - **Accuracy** (tačnost)  
     - **Precision**, **Recall**, **F1 Score**  
   - Izabran je model Random Forest sa najboljim balansom između tačnosti i jednostavnosti.  


In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline

# Izabrani modeli koji ce se koristiti za treniranje
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Support Vector Classifier': SVC()
}

# Treniranje sva 3 modela i provjera tacnosti
for name, model in models.items():
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    print(f"=== {name} ===")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print("\n")


=== Logistic Regression ===
Accuracy: 0.9478632478632478
                  precision    recall  f1-score   support

             CPU       0.00      0.00      0.00        17
            CPUs       0.98      1.00      0.99       749
 Digital Cameras       0.99      0.99      0.99       538
     Dishwashers       0.93      0.96      0.94       681
        Freezers       0.98      0.91      0.94       440
 Fridge Freezers       0.93      0.94      0.93      1094
         Fridges       0.87      0.88      0.88       687
      Microwaves       0.98      0.95      0.97       466
    Mobile Phone       0.00      0.00      0.00        11
   Mobile Phones       0.97      0.99      0.98       801
             TVs       0.97      0.99      0.98       708
Washing Machines       0.94      0.95      0.94       803
          fridge       0.00      0.00      0.00        25

        accuracy                           0.95      7020
       macro avg       0.73      0.73      0.73      7020
    weighted 

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


=== Random Forest ===
Accuracy: 0.949002849002849
                  precision    recall  f1-score   support

             CPU       0.00      0.00      0.00        17
            CPUs       0.98      0.99      0.98       749
 Digital Cameras       1.00      0.97      0.98       538
     Dishwashers       0.96      0.95      0.95       681
        Freezers       0.92      0.92      0.92       440
 Fridge Freezers       0.92      0.95      0.93      1094
         Fridges       0.89      0.88      0.89       687
      Microwaves       0.95      0.97      0.96       466
    Mobile Phone       0.00      0.00      0.00        11
   Mobile Phones       0.95      0.99      0.97       801
             TVs       0.99      0.99      0.99       708
Washing Machines       0.97      0.94      0.96       803
          fridge       0.00      0.00      0.00        25

        accuracy                           0.95      7020
       macro avg       0.73      0.73      0.73      7020
    weighted avg    

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


6. **Spremanje modela sa joblib bibliotekom**

In [34]:
import joblib
from google.colab import files

# Uzimamo RandomForestClassifier model
best_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Trenira se na cijelom trening setu
best_pipeline.fit(X_train, y_train)

# Spremanje u lokalni fajl
joblib.dump(best_pipeline, 'product_classifier.pkl')

# Spremanje fajla na racunar
files.download('product_classifier.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>